In [ ]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility

# importlib.reload(db_info)

In [ ]:
#!pip install py_vollib

In [ ]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdvals,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=go.Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    return fig


In [ ]:
pga = db_info.get_db_info()
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'


### get data from sql 

In [ ]:
osql = f"select * from {opttab} where substring(settle_date::text,1,5)='20150';"
dfo = pga.get_sql(osql)
usql = f"select * from {futtab} where substring(settle_date::text,1,5)='20150';"
dfu = pga.get_sql(usql)


In [ ]:
print(dfo.shape,dfu.shape)
display.display(dfo.head())
display.display(dfu.head())

In [ ]:
df = dfo.merge(dfu,how='inner',on=['symbol','settle_date'])

In [ ]:
print(df.shape)
df.head()

### create pivot of strikes per day

In [ ]:

df2 = df[['symbol','contract_num','pc','settle_date','strike','close_x','close_y']]
df2 = df2[(df2.pc=='C') & (df2.symbol.str.slice(0,2).isin(['CL']))]
df2 = df2[df2.contract_num==2]
df2.strike = df2.strike/10
phigh = df2.close_y.max()
plow = df2.close_y.min()
high_strike = round(phigh * 1.2)
low_strike = round(plow * .8)
print(high_strike,low_strike,df2.strike.max(),df2.strike.min())
df2 = df2[(df2.strike>=low_strike) & (df2.strike<=high_strike)]
df2['symset'] = df2.symbol + "_" + df2.settle_date.astype(str)
df3 = df2.pivot(index='symset',columns='strike',values='close_x')
df3['symset'] = df3.index.values
df3.index.name=None
df3.index = list(range(len(df3)))
df3.columns.name = None
df3['symset_array'] = df3.symset.str.split('_')
df3['symbol'] = df3.symset_array.apply(lambda a:a[0])
df3['settle_date'] = df3.symset_array.apply(lambda a:a[1])
cols_in_order = ['symbol','settle_date'] + [c for c in df3.columns.values if c not in ['symbol','settle_date','symset','symset_array']]
df3 = df3[cols_in_order]
df3.settle_date = df3.settle_date.astype(str).astype(int)
df3.head()                                     

In [ ]:

mcols = ['symbol','settle_date']
df4 = dfu[mcols + ['close']]
df5 = df3.merge(df4,on=mcols,how='inner')
first_cols = ['symbol','settle_date','close']
cols_in_order = first_cols + [c for c in df5.columns.values if c not in first_cols]
df5 = df5[cols_in_order]
df5
                                                          

In [ ]:
w1 = "char_length(symbol)=5 and substring(symbol,1,2)='CL' and substring(symbol,4,2)::int>='15'"

osql = f"select * from {opttab} where {w1};"
dfo2 = pga.get_sql(osql)


In [ ]:
df_expiry_dates = dfo2[['symbol','settle_date']].groupby('symbol',as_index=False).max()

In [ ]:
df_expiry_dates.head()

In [ ]:
df5['expiry'] = df5[['symbol']].merge(df_expiry_dates,on='symbol',how='inner').settle_date
df6 = df5.copy()
df6['syear'] = df6.settle_date.astype(str).str.slice(0,4).astype(int)
df6['smon'] = df6.settle_date.astype(str).str.slice(4,6).astype(int)
df6['sday'] = df6.settle_date.astype(str).str.slice(6,8).astype(int)
df6['eyear'] = df6.expiry.astype(str).str.slice(0,4).astype(int)
df6['emon'] = df6.expiry.astype(str).str.slice(4,6).astype(int)
df6['eday'] = df6.expiry.astype(str).str.slice(6,8).astype(int)
df6['sdatetime'] = df6.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
df6['edatetime'] = df6.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
df6['dte'] = df6.edatetime - df6.sdatetime
df6.dte = df6.dte.dt.days
on_cols = ['symbol','settle_date']
df5['dte'] = df5[on_cols].merge(df6[on_cols+['dte']],on=on_cols,how='inner').dte

In [ ]:
df5.head()

In [ ]:
non_strikes = ['symbol','settle_date','close','expiry','dte']
strikes = list(set(df5.columns.values).difference(set(non_strikes)))
df6 = df5.copy()
df6 = df6.rename(columns = {c:str(c)+'s' for c in strikes})
strikes = list(set(df6.columns.values).difference(set(non_strikes)))
r = df6.iloc[0]
opt_info = [r.close,float(strikes[0][:-1]),2,r.dte]
r['close'],r[strikes[0]]
mibian.BS(opt_info, callPrice=r[strikes[0]]).impliedVolatility


#### get implied vols

In [ ]:
non_strikes = ['symbol','settle_date','close','expiry','dte']
strikes = list(set(df5.columns.values).difference(set(non_strikes)))
df6 = df5.copy()
df6 = df6.rename(columns = {c:str(c)+'s' for c in strikes})
strikes = list(set(df6.columns.values).difference(set(non_strikes)))

print(strikes)
for strike in tqdm_notebook(strikes):
    s = float(strike.replace('s',''))
    df6[f"{s}c"] = df6[non_strikes + [strike]].apply(
        lambda r: -1 if np.isnan(r[strike]) else mibian.BS([r.close,s,2,r.dte], callPrice=r[strike]).impliedVolatility
        ,axis=1)
    

In [ ]:
df7 = df6[non_strikes + [c.replace('s','c') for c in strikes]]

In [ ]:
df7.head()

#### get deltas

In [ ]:
df8 = df7.copy()
non_strikes = ['symbol','settle_date','close','expiry','dte']
strikes = list(set(df8.columns.values).difference(set(non_strikes)))

print(strikes)
for strike in tqdm_notebook(strikes):
    s = float(strike.replace('c',''))
    df8[f"{s}c"] = df8[non_strikes + [strike]].apply(
        lambda r: -1 if np.isnan(r[strike]) else mibian.BS([r.close,s,2,r.dte], volatility=r[strike]).callDelta
        ,axis=1)


In [ ]:
df8.head()

#### do implied vol interpolation to find vol at option that is 10% out of the money

In [ ]:
non_strikes = ['symbol','settle_date','close','expiry','dte']
strikes = list(set(df7.columns.values).difference(set(non_strikes)))
dft =df7[strikes].iloc[:1]
dft = dft.T.sort_index()
dft.index = [float(s.replace('c','')) for s in dft.index]
dft.columns = ['imp']
dft.head()

In [ ]:
a

In [ ]:
v = 51.5
a = np.arange(.7,1.3,.05) * v
a = a.round(6)
for x in a:
    dft.loc[x,'imp'] = None
dft = dft.sort_index()
dft['imp2'] = dft.imp.interpolate(method='polynomial', order=2)
dft['skew'] = (dft.imp2 - dft.loc[v].imp2).round(4)
dft['money'] = (dft.index / v - 1)
dft.money = dft.money.round(4)
dft[dft.index.isin(a)]
# a = np.array(sorted([float(s.replace('c','')) for s in strikes]))
# b = 
# m = np.abs(a - t).argmin()
# pair =(a[m],a[m+1]) if a[m]<t else (a[m-1],a[m])

# print(t,pair)
# pd.DataFrame({'a':a})


In [ ]:
display(df2.head())
display(df_expiry_dates.head())

In [ ]:
#CL,Q2019,560P,07/02/2019,0.6,1.61,0.54,1.54,1997,4465
F = 56.25
K = 56
sigma = .366591539
flag = 'p'
t = 15/365.0
r = .025
discounted_call_price = black.black(flag, F, K, t, r, sigma)
dcp = 1.54
ivpy = implied_volatility.implied_volatility(dcp, F, K, r, t, flag)
ivmn = mibian.BS([F,K,2.5,15], callPrice=dcp).impliedVolatility
discounted_call_price,ivpy,ivmn


In [ ]:
USE_PYVOL = True
contract='CLM15'
df9 = df2[df2.symbol==contract]
df9 = df9.merge(df_expiry_dates.rename(columns={'settle_date':'expiry'}),on='symbol',how='inner')
df9['syear'] = df9.settle_date.astype(str).str.slice(0,4).astype(int)
df9['smon'] = df9.settle_date.astype(str).str.slice(4,6).astype(int)
df9['sday'] = df9.settle_date.astype(str).str.slice(6,8).astype(int)
df9['eyear'] = df9.expiry.astype(str).str.slice(0,4).astype(int)
df9['emon'] = df9.expiry.astype(str).str.slice(4,6).astype(int)
df9['eday'] = df9.expiry.astype(str).str.slice(6,8).astype(int)
df9['sdatetime'] = df9.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
df9['edatetime'] = df9.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
df9['dte'] = df9.edatetime - df9.sdatetime
df9.dte = df9.dte.dt.days
df9 = df9[['symbol','settle_date','pc','contract_num','strike','close_x','close_y','dte']]
df10 = df9.iloc[:len(df9)]
df10.index = list(range(len(df10)))
lam_pyvol = lambda r:implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
lam_mibian = lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility
n = 100
for i in tqdm_notebook(np.arange(0,len(df10)-n,n)):
    if USE_PYVOL:
        df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_pyvol,axis=1)
    else:
        df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_mibian,axis=1)
#     ivv = implied_volatility.implied_volatility(discounted_call_price, F, K, r, t, flag)    
#     df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility,axis=1)
    
print(f'doing remaining {datetime.datetime.now()}')
i = df10[df10.iv.isna()].index[0]
# df10.loc[i:,'iv'] = df10.loc[i:].apply(lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility,axis=1)
if USE_PYVOL:
    df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_pyvol,axis=1)
else:
    df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_mbian,axis=1)

print(f'doing remaining {datetime.datetime.now()}')
display.display(df10.head())
display.display(df10.tail())

#100% 13/13 [00:41<00:00, 3.20s/it]


In [ ]:
# df10_save_mbian = df10.copy()
# df10_save_pyvol = df10.copy()
print(df10_save_mbian.shape,df10_save_pyvol.shape)
display.display(df10_save_mbian.head())
display.display(df10_save_pyvol.head())



In [ ]:
# define amounts around the money which will help create strikes to add
moneyness = np.arange(.7,1.4,.05).round(6)
# define columns on which to execute groupby
gb_cols = ['symbol','settle_date','pc','contract_num','dte','close_y']
# define function used in groupby.apply to create strikes and iv's at those strikes
#   where the strikes are an even amount from the money 
#   (like .7, .8, ... 1, 1.1, 1.2, etc)
def _add_even_money_strikes(df):
    # get underlying from first row (the groupby makes them all the same)
    r = df.iloc[0]
    underlying = r.close_y
    # create new rows to append to df, using only the gb_cols
    df_ret1 = df.iloc[:len(moneyness)][gb_cols]
    # add nan iv's !!!! MUST BE np.nan - NOT None
    df_ret1['iv'] = np.nan
    # add new strikes
    df_ret1['strike'] = moneyness * underlying
    # append the new strikes
    df_ret2 = df.append(df_ret1,ignore_index=True).sort_values(['symbol','settle_date','pc','strike'])
    df_ret2 = df_ret2.drop_duplicates(subset='strike')
    # set the index to the strike so that interpolate works
    df_ret2.index = df_ret2.strike
    # create interpolated iv's
    df_ret2['iv'] = df_ret2.iv.interpolate(method='polynomial', order=2)
    # reset the index
    df_ret2.index = list(range(len(df_ret2)))
    return df_ret2


df11 = df10.groupby(gb_cols).apply(_add_even_money_strikes)
df11.index = list(range(len(df11)))
df11['moneyness'] = (df11.strike / df11.close_y).round(6)

df12 = df11[(df11.moneyness.isin(moneyness)) & (~df11.iv.isna())]
df12.moneyness  = df12.moneyness - 1
df12.index = list(range(len(df12)))
df12_atm = df12[df12.moneyness==0][['symbol','settle_date','pc','iv']]
df12_atm = df12_atm.rename(columns={'iv':'atm_iv'})
df12_atm = df12_atm.drop_duplicates()
df12 = df12.merge(df12_atm,on=['symbol','settle_date','pc'],how='inner')
df12.moneyness = df12.moneyness.round(6)
df12['vol_skew'] = (df12.iv - df12.atm_iv).round(4)
df12[['symbol','settle_date','moneyness','vol_skew']]


In [ ]:
df12[df12.settle_date==20150327]

### graph skew

In [ ]:
settle_dates =  sorted(df12.settle_date.unique())
first_settle_date = settle_dates[0]
dfp = df12[df12.settle_date==first_settle_date][['moneyness','vol_skew']]
dfp = dfp.rename(columns={'vol_skew':str(first_settle_date)})
i=0
for s in settle_dates[1:]:
    dfp_this_s = df12[df12.settle_date==s][['moneyness','vol_skew']]
    dfp = dfp.merge(dfp_this_s,on='moneyness').rename(columns={'vol_skew':str(s)})
#     i+=1
#     if i>3:
#         break
display.display(dfp)
splits = list(np.arange(5,len(settle_dates),5))
settle_date_strings = [str(yyyymmdd) for yyyymmdd in settle_dates]
settle_date_groups = np.split(np.array(settle_date_strings),splits)
for sdg in settle_date_groups:
    cols = ['moneyness']+list(sdg)
    dfp_sub = dfp[cols]
    t = f'{contract} {list(set([c[:-2] for c in sdg]))}'
    iplot(plotly_plot(dfp_sub,x_column='moneyness',plot_title=t))

In [ ]:
for yyyymmdd in sorted(df12.settle_date.unique()):
    dfp = df12[df12.settle_date==yyyymmdd][['moneyness','skew']]
    iplot(plotly_plot(dfp,x_column='moneyness',plot_title=str(yyyymmdd)))

In [ ]:
# s = ['symbol','pc','settle_date','strike']
# dft = df10.append(df11,ignore_index = True).sort_values(s)
# dft['iv2'] = dft.iv.interpolate(method='polynomial', order=2)

# dft['skew'] = (dft.iv2 - dft.loc[v].imp2).round(4)
# dft['money'] = (dft.strike / dft.close_y - 1)
# dft.money = dft.money.round(4)
# dft[dft.index.isin(a)]


In [ ]:
underlying=1.4565
strike=1.45
interest = 1
days=30
opt_info = [underlying,strike,interest,days]
c = mibian.BS(opt_info, volatility=20)
print(c.callPrice,
c.putPrice,
c.callDelta,
c.putDelta,
c.callDelta2,
c.putDelta2,
c.callTheta,
c.putTheta,
c.callRho,
c.putRho,
c.vega,
c.gamma)


co = mibian.BS(opt_info, callPrice=c.callPrice)
co.impliedVolatility